In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import data_processing as dp
from scipy import signal
from scipy.stats import skew
import seaborn as sns
import matplotlib.pyplot as plt
import pywt
import os
import time
import h5py
from itertools import combinations

In [ ]:
for dirname, _, filenames in os.walk('.\data'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

In [ ]:
path2 = '.\data\正常\G11_Walking_trial_4_emg.csv'

In [56]:
Data = pd.read_csv('./processed data/dataframe_W256_S64_R180_DWTLmax_dropna_samelabel_MF3.csv')

# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
drop = 'P379_M050_2_OFF_A_FoG_trial_2_emg.csv'
drop2= 'P812_M050_2_B_FoG_trial_2_emg.csv'
ind_drop = (df.columns!=drop) | (df.columns!=drop2)
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)

files = np.concatenate([np.array(df.columns[ind_drop]),np.array(df3.loc[:,0])])
ind = Data.File.isin(files)
#ind = (Data.File != drop) & (Data.File != drop2)
Data_sel = Data[ind]
Data_rest = Data[~ind]
#ind2 = Data_rest.File == drop
#Data_rest = Data_rest[ind2]
#Data_rest

In [57]:
feature_col = ['_IEMG','_MAV','_SSI','_VAR','_RMS',
               '_WL','_ZC','_SSC','_WAMP','_skew','_Acti','_AR','_HIST','_MDF']

feature_all = Data_sel.iloc[:,1:-1]
#ind_temp1 = feature_all.columns.str.contains('_mDWT')
#ind_temp2 = feature_all.columns.str.contains('_Coe')
#ind_temp3 = feature_all.columns.str.contains('_Scale')
#ind_temp4 = feature_all.columns.str.contains('_HIST')
#ind_temp = ind_temp1|ind_temp2|ind_temp3|ind_temp4
ind_temp = feature_all.columns.str.contains('_MF')
feature = feature_all.loc[:,~ind_temp]
#feature = Data_sel.iloc[:,1:-1]
#feature = Data_sel.iloc[:,:-2]
y = Data_sel.Label
#feature2_all = Data_rest.iloc[:,1:-1]
#feature2 = feature2_all.loc[:,ind_temp]
feature2 = Data_rest.iloc[:,1:-1]
#feature2 = Data_rest.iloc[:,:-2]
y2 = Data_rest.Label

In [ ]:
path = './processed data/data_set_after_window_withoutSC.hdf5'
with h5py.File(path,'r') as f:
    x = f['cwt_data'][...]
    y = f['label2'][...]

In [341]:
import imp
imp.reload(dp)

<module 'data_processing' from 'E:\\Document\\jupyter\\Master Thesis\\data_processing.py'>

In [3]:
import ctypes
player = ctypes.windll.kernel32
player.Beep(1000,200)

1

In [52]:
feature

,LEFT_TA_IEMG,LEFT_TS_IEMG,LEFT_BF_IEMG,LEFT_RF_IEMG,RIGHT_TA_IEMG,RIGHT_TS_IEMG,RIGHT_BF_IEMG,RIGHT_RF_IEMG,LEFT_TA_SSI,LEFT_TS_SSI,...,RIGHT_BF_mDWT6,RIGHT_BF_mDWT7,RIGHT_RF_mDWT0,RIGHT_RF_mDWT1,RIGHT_RF_mDWT2,RIGHT_RF_mDWT3,RIGHT_RF_mDWT4,RIGHT_RF_mDWT5,RIGHT_RF_mDWT6,RIGHT_RF_mDWT7
0,548.09960,497.49110,681.18840,598.77924,1313.22860,4442.02050,1284.18550,693.58180,2070.7683,1748.4358,...,9.377046,4.990916,556.434814,343.095306,207.635269,93.610168,53.094650,24.570675,12.060508,5.739622
1,543.63100,484.05870,665.12604,638.29640,1035.78380,3225.43430,1263.43310,687.76100,2057.3930,1713.7683,...,26.621002,14.194582,505.209656,279.325409,165.125229,83.372551,43.600906,20.352535,9.021744,4.887817
2,524.71530,491.95126,639.57263,603.66766,990.65460,3633.17200,1217.76420,628.88730,1964.1893,1775.8363,...,38.259026,19.512487,425.959656,226.959534,119.487320,45.126644,8.551785,4.179543,1.943067,0.781084
3,547.05853,466.00555,628.01060,632.10790,985.55725,5042.98700,1205.29320,592.94604,2096.5870,1575.4939,...,25.216244,11.351563,395.746338,213.186203,88.206375,41.611679,4.594048,1.401577,0.310067,0.121287
4,515.43780,446.27258,626.81300,581.82970,869.83300,3146.06740,1249.10460,610.60790,2019.2968,1537.0039,...,8.140111,4.368567,477.853790,305.213226,166.140747,106.498840,68.548775,30.848173,15.781103,7.525197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16101,2210.39600,2431.59250,2522.21500,651.74896,2351.49150,909.37680,583.51416,869.30070,65793.2400,61341.7580,...,16.480499,7.987933,679.553101,465.168945,255.410934,123.197403,71.788757,31.097105,16.020678,7.656378
16102,2862.80100,2530.60000,2776.12740,645.07587,2642.48320,1071.73520,589.38550,805.04430,117425.8700,66555.9400,...,15.043890,7.118279,611.477295,414.233459,235.653336,101.961647,48.022587,23.091249,9.786898,5.183936
16103,2109.35350,3724.72560,2676.46920,630.86170,2719.53600,976.24420,611.96580,830.08466,86317.1000,120090.4700,...,10.769453,4.862518,669.540161,473.558655,263.836212,112.255241,57.610939,24.506268,12.162224,5.727070
16104,3429.48580,2834.17070,3254.47530,676.47100,2421.15940,899.85250,590.83167,718.16504,170800.9500,86476.3500,...,25.024672,11.816714,493.144226,299.594940,163.800293,76.252205,42.552967,19.244778,9.842437,4.149382


In [ ]:
width = 256
threshold_WAMP = 30
threshold_ZC = 0
threshold_SSC = 1
bins=9
bound = 70
HIST_range = (-bound,bound)

In [4]:
path = './processed data/data_set_after_window_W128_S64_sameLabel_rect.hdf5'
with h5py.File(path,'r') as f:
    x = f['data'][...]
    y = f['label2'][...]
#feature = dp.generate_feature(x,threshold_WAMP=threshold_WAMP,
#                              threshold_ZC=threshold_ZC,
#                              threshold_SSC=threshold_SSC,
#                              bins=bins,ranges=HIST_range)
#feature2 = dp.generate_feature(x2)
player.Beep(1000,200)

1

In [ ]:
with h5py.File('./processed data/nfeatures_W256_S64_WAMP30.hdf5','r') as f:
    feature = f['features'][...]
    y = f['labels'][...]

In [ ]:
feature2,y2 = dp.pipeline_feature(path2,width=256,stride=64,
                                  scaler=False,
                                  threshold_WAMP=threshold_WAMP,
                                  threshold_ZC=threshold_ZC,
                                  threshold_SSC=threshold_SSC,
                                  bins=bins,ranges=HIST_range)

In [ ]:
with h5py.File('./processed data/nfeatures_rest_W256_S64.hdf5','r') as f:
    feature2 = f['features'][...]
    y2 = f['labels'][...]

In [ ]:
sc = StandardScaler(with_mean=True)
feature_sc = sc.fit_transform(feature)
feature2_sc = sc.transform(feature2)

# Model

In [4]:
import tensorflow as tf
from tensorflow.keras import layers,Model,callbacks,regularizers,models
from sklearn.model_selection import train_test_split,cross_validate
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder,normalize,MinMaxScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE,BorderlineSMOTE,ADASYN,SVMSMOTE

In [5]:
tf.executing_eagerly()

True

In [51]:
def train_model(model,feature,y,binary=True,file=None):
    if binary:
        ind = ((y==0)|(y==1)|(y==2)|(y==3)|(y==4)|(y==6))
        ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y.copy()
        #ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
        y_01[ind1] = 1
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:5}
    else:
        ind = ((y==1)|(y==2)|(y==6))
        #ind = ((y==1)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:5,1:1,2:1}#{0:2,1:1,2:10,3:2}
    x_full,x_test,y_full,y_test = train_test_split(np.array(feature)[ind,:],y_01,
                                                   test_size=0.2,
                                                   random_state=123,
                                                   shuffle=True)
    x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,
                                                       test_size=0.25,
                                                       random_state=555,
                                                       shuffle=True)
    
    #sm = BorderlineSMOTE(random_state=50,kind='borderline-2')
    #sm = SMOTE(random_state=50)
    #print(y_full.shape)
    #x_full,y_full = sm.fit_resample(x_full,y_full)
    #print(y_full_n.shape)
    sc = StandardScaler(with_mean=True)
    #sc = MinMaxScaler()
    x_train = sc.fit_transform(x_full)
    #pca = PCA(n_components=100)
    #x_train = pca.fit_transform(x_train)
    #x_valid = sc.transform(x_valid)
    x_test = sc.transform(x_test)
    #x_test = pca.transform(x_test)
    #x_train = x_full
    
    early_stopping = callbacks.EarlyStopping(patience = 20,
                                             monitor = 'val_loss', 
                                             restore_best_weights=True)
    history = model.fit(x_train,y_full,validation_data=[x_test,y_test],
                        epochs=300,batch_size=32,class_weight=cw,
                        callbacks=[early_stopping],
                        shuffle=True)
    
    if binary:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #test = metrics.accuracy_score(y_test,y_pred_t>0.5)
        
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(y_valid,np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))
        #train = metrics.accuracy_score(y_full,y_pred_ta>0.5)
        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
        
        #print('train: \n',metrics.confusion_matrix(y_full,y_pred_ta>0.5))
        #print('test: \n',metrics.confusion_matrix(y_test,y_pred_t>0.5))

    else:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
    print('test:%f'%test)
    #print('valid:%f'%valid)
    print('train:%f'%train)
    if file != None:
        model.save_model(file)
    return train,test,sc#,pca

def test_model(model,feature,y,sc,pca):
    ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
    #ind = ((y==0)|(y==1)|(y==2)|(y==6))
    y_01 = y.copy()
    #ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
    y_01[ind] = 1
    #print(set(y_01))
    oh_ec = OneHotEncoder()
    y_01 = oh_ec.fit_transform(y_01[:,np.newaxis]).toarray()
    #print(y_01)
    feature=sc.transform(feature)
    #feature=pca.transform(feature)
    y_pred=model.predict(feature)
    test = metrics.accuracy_score(np.argmax(y_01,axis=1),np.argmax(y_pred,axis=1))
    #test = metrics.accuracy_score(y_01,y_pred>0.5)
    
    print('acc:%f'%test)
    print(metrics.confusion_matrix(np.argmax(y_01,axis=1),np.argmax(y_pred,axis=1)))
    #print(metrics.confusion_matrix(y_01,y_pred>0.5))
    return test

def sparse_cost_sensitive_loss (y_true,y_pred):
    #cost_matrix = tf.constant([[0,1.,1,1.],
    #              [2,0,5,5],
    #              [1,1,0,1],
    #              [1.,2.,1,0]])
    cost_matrix = tf.constant([[0,2.,2],
                  [1,0,1],
                  [1.0,1.,0]])
    #cost_matrix = tf.constant([[0,2.],
    #              [2.,0]])
    batch_cost_matrix = tf.nn.embedding_lookup(cost_matrix, tf.argmax(y_true,axis=1))
    eps = 1e-6
    probability = tf.clip_by_value(y_pred, eps, 1-eps)
    cost_values = tf.math.log(1-probability)*batch_cost_matrix
    loss = tf.reduce_mean(-tf.reduce_sum(cost_values, axis=1))
    return loss

## ANN

In [52]:
ind = ((y==1)|(y==2)|(y==3)|(y==6))
#ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
ind_f = [0,1,6,42,46,57,62]
#y_01 = y[ind].copy()
#y_01[y_01==1]=0
#y_01[y_01==2]=1
#y_01[y_01==3]=2
#y_01[y_01==6]=3
y_01 = y[ind].copy()
#y_01[ind] = 1
oh_ec = OneHotEncoder()
y_oh = oh_ec.fit_transform(y_01[:,np.newaxis]).toarray()
x_full,x_test,y_full,y_test = train_test_split(feature.loc[ind,:],y_01,test_size=0.2,random_state=123,shuffle=False)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555,shuffle=True)

In [53]:
input_ = layers.Input(shape=feature.shape[1:])#feature.shape[1:]
l1 = layers.Dense(128,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(input_)
#drop1 = layers.Dropout(0.2)(l1)
bn1 = layers.BatchNormalization()(l1)
l2 = layers.Dense(64,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(bn1)
#drop2 = layers.Dropout(0.2)(l2)
bn2 = layers.BatchNormalization()(l2)
l3 = layers.Dense(32,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(bn2)
#drop3 = layers.Dropout(0.2)(l3)
bn3 = layers.BatchNormalization()(l3)
l4 = layers.Dense(16,activation='elu',
                 #kernel_regularizer = regularizers.l2(0.001),
                 )(bn3)
#drop4 = layers.Dropout(0.2)(l4)
#l5 = layers.Dense(8,activation='relu')(drop4)
#drop5 = layers.Dropout(0.5)(l5)
output = layers.Dense(3,activation='softmax')(l4)
model = Model(inputs=[input_],outputs=[output])

In [58]:
model = models.Sequential()
model.add(layers.Dense(128,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 ))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 ))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(32,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 ))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(16,activation='elu',
                 #kernel_regularizer = regularizers.l2(0.001),
                 ))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(3,activation='softmax'))

In [59]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train model

In [60]:
#sc = StandardScaler(with_mean=True)
#feature_sc = sc.fit_transform(feature)
#feature2_sc = sc.transform(feature2)
#pca = PCA(n_components=160,copy=True)
#feature_pca = pca.fit_transform(feature)
#feature2_pca = pca.transform(feature2)
train,test,sc = train_model(model,feature,np.array(y),False)
#rest = test_model(model,feature2,np.array(y2),sc)
#acc = [train,valid,test,rest]

E:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Train on 12810 samples, validate on 2132 samples
Epoch 1/300
12810/12810 [==============================] - 5s 360us/sample - loss: 0.8787 - accuracy: 0.5855 - val_loss: 0.6497 - val_accuracy: 0.7308
Epoch 2/300
12810/12810 [==============================] - 1s 87us/sample - loss: 0.7899 - accuracy: 0.6340 - val_loss: 0.5930 - val_accuracy: 0.7425
Epoch 3/300
12810/12810 [==============================] - 1s 88us/sample - loss: 0.7316 - accuracy: 0.6693 - val_loss: 0.6297 - val_accuracy: 0.7312
Epoch 4/300
12810/12810 [==============================] - 1s 85us/sample - loss: 0.7001 - accuracy: 0.6826 - val_loss: 0.5788 - val_accuracy: 0.7580
Epoch 5/300
12810/12810 [==============================] - 1s 84us/sample - loss: 0.6734 - accuracy: 0.6920 - val_loss: 0.5846 - val_accuracy: 0.7453
Epoch 6/300
12810/12810 [==============================] - 1s 88us/sample - loss: 0.6495 - accuracy: 0.7037 - val_loss: 0.5810 - val_accuracy: 0.7570
Epoch 7/300
12810/12810 [=========================

Epoch 55/300
12810/12810 [==============================] - 1s 73us/sample - loss: 0.2842 - accuracy: 0.8931 - val_loss: 0.4638 - val_accuracy: 0.8358
Epoch 56/300
12810/12810 [==============================] - 1s 72us/sample - loss: 0.2681 - accuracy: 0.8959 - val_loss: 0.4813 - val_accuracy: 0.8335
Epoch 57/300
12810/12810 [==============================] - 1s 73us/sample - loss: 0.2714 - accuracy: 0.8955 - val_loss: 0.4792 - val_accuracy: 0.8326
Epoch 58/300
12810/12810 [==============================] - 1s 73us/sample - loss: 0.2612 - accuracy: 0.8998 - val_loss: 0.4844 - val_accuracy: 0.8297
Epoch 59/300
12810/12810 [==============================] - 1s 77us/sample - loss: 0.2757 - accuracy: 0.8967 - val_loss: 0.4587 - val_accuracy: 0.8415
Epoch 60/300
12810/12810 [==============================] - 1s 73us/sample - loss: 0.2636 - accuracy: 0.8977 - val_loss: 0.4933 - val_accuracy: 0.8480
Epoch 61/300
12810/12810 [==============================] - 1s 73us/sample - loss: 0.2678 - ac

In [ ]:
test_model(model,np.array(feature2),np.array(y2),sc,pca)

In [230]:
pd.Series(y2).value_counts()

0    460
2    430
1     33
3      8
Name: Label, dtype: int64

In [ ]:
#test_model(model,feature2_sc,np.array(y2))
ind = ((y==1)|(y==2)|(y==6))
y_01 = y[ind].copy()
oc = OneHotEncoder()
y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
pred=model.predict(feature_sc[ind,:])
np.argmax(pred,axis=1)
np.argmax(y_01,axis=1)
metrics.confusion_matrix(np.argmax(y_01,axis=1),np.argmax(pred,axis=1))

In [ ]:
#acc_ann = pd.DataFrame(acc,columns=['dnn1'],index=['train','valid','test','rest data'])
#acc_ann_com=pd.concat([acc_ann_com,acc_ann.T])
#acc_ann.loc['drop_mDWT',:]=[train,valid,test,rest]
acc_ann_com.loc['dnn2',:]=acc
acc_ann_com

# test feature from Left or Right

In [ ]:
input_ = layers.Input(shape=feature.loc[:,ind_temp].shape[1:])
l1 = layers.Dense(128,activation='elu')(input_)
drop1 = layers.Dropout(0.2)(l1)
l2 = layers.Dense(64,activation='elu')(drop1)
drop2 = layers.Dropout(0.2)(l2)
#l3 = layers.Dense(32,activation='elu')(drop2)
#drop3 = layers.Dropout(0.2)(l3)
#l4 = layers.Dense(16,activation='selu')(l3)
#drop4 = layers.Dropout(0.2)(l4)
output = layers.Dense(1,activation='sigmoid')(drop2)
model = Model(inputs=[input_],outputs=[output])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [209]:
acc={}
cols = ['LEFT','RIGHT']

#sc = StandardScaler(with_mean=True)

for col in cols:
    ind_temp=feature.columns.str.contains(col)
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    #acc[col] = [train,valid,test,acc_rest]
    print(acc_rest)

Train on 23125 samples, validate on 5782 samples
Epoch 1/200
23125/23125 [==============================] - 4s 167us/sample - loss: 1.9657 - accuracy: 0.5308 - val_loss: 1.3537 - val_accuracy: 0.6404
Epoch 2/200
23125/23125 [==============================] - 3s 109us/sample - loss: 1.3913 - accuracy: 0.6479 - val_loss: 1.1555 - val_accuracy: 0.7008
Epoch 3/200
23125/23125 [==============================] - 3s 110us/sample - loss: 1.2045 - accuracy: 0.7052 - val_loss: 1.0439 - val_accuracy: 0.7771
Epoch 4/200
23125/23125 [==============================] - 2s 105us/sample - loss: 1.1076 - accuracy: 0.7394 - val_loss: 0.9467 - val_accuracy: 0.7608
Epoch 5/200
23125/23125 [==============================] - 2s 86us/sample - loss: 1.0526 - accuracy: 0.7501 - val_loss: 0.9212 - val_accuracy: 0.7508
Epoch 6/200
23125/23125 [==============================] - 2s 93us/sample - loss: 0.9990 - accuracy: 0.7677 - val_loss: 0.8937 - val_accuracy: 0.7975
Epoch 7/200
23125/23125 [======================

23125/23125 [==============================] - 2s 71us/sample - loss: 0.8517 - accuracy: 0.8045 - val_loss: 0.8904 - val_accuracy: 0.8210
Epoch 9/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.8091 - accuracy: 0.8131 - val_loss: 1.0212 - val_accuracy: 0.8426
Epoch 10/200
23125/23125 [==============================] - 2s 72us/sample - loss: 0.7750 - accuracy: 0.8260 - val_loss: 1.0699 - val_accuracy: 0.8438
Epoch 11/200
23125/23125 [==============================] - 2s 72us/sample - loss: 0.7591 - accuracy: 0.8365 - val_loss: 0.9963 - val_accuracy: 0.8513
Epoch 12/200
23125/23125 [==============================] - 2s 70us/sample - loss: 0.7071 - accuracy: 0.8416 - val_loss: 1.0759 - val_accuracy: 0.8464
Epoch 13/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.6955 - accuracy: 0.8444 - val_loss: 1.0100 - val_accuracy: 0.8442
Epoch 14/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.7017 - accuracy: 0.8478

In [ ]:
acc_lr=pd.DataFrame(acc,index=['train','valid','test','rest data'])#.to_csv('./results/acc_lr_ann.csv')
#acc_com=pd.concat([acc_ann_com.drop(['dnn','dnn1'],'index'),acc_lr.T])
acc_com

# test features from 2 of 8 signals

In [ ]:
#test features from 2 of 8 signals

acc_2f={}
cols = [ 'LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF',
        'RIGHT_TA','RIGHT_TS','RIGHT_BF', 'RIGHT_RF']


for p in combinations(cols[:4],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)
    
for p in combinations(cols[4:],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)

In [ ]:
#acc_2f_ann = pd.DataFrame(acc_2f,index=['train','valid','test','rest data']).T
#acc_com = pd.concat([acc_com,acc_2f_ann])
acc_com.to_csv('./results/dropna/acc_com_ann.csv')

In [ ]:
pd.DataFrame(acc_2f,index=['train','valid','test','rest data']).T

# Test on some of rest

In [ ]:
some_of_rest = ['正常/P940_MSham_B_Walking_trial_6_emg.csv',
                '正常/P940_M050_B_Walking_trial_4_emg.csv',
                '正常/P812_M100_A_Walking_trial_3_emg.csv',
                '正常/P645_M050_A_Walking_trial_3_emg.csv',
                '正常/P623_Msham_B_Walking_trial_2_emg.csv',
                '正常/P551_M50_B_Walking_trial_6_emg.csv',
                'P379_M050_2_OFF_A_FoG_trial_1_emg.csv',
                'P551_M050_2_B_FoG_trial_2_emg.csv']
#booster = xgb.Booster()
#booster.load_model('./model/XGBoost_W256_S64_Left.json')
#model = xgb.XGBClassifier()
#model._Booster = booster
acc = []
columns=['LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF']
for file in some_of_rest:
    path = './data/'+file
    feature2,y2 = dp.pipeline_feature(path,width=256,stride=64,scaler=False,
                                      threshold_WAMP=threshold_WAMP,
                                      threshold_ZC=threshold_ZC,
                                      threshold_SSC=threshold_SSC,
                                      bins=bins,
                                      ranges=HIST_range,
                                      show_para=False,
                                      filt = 250)
    feature2_sc = sc.transform(feature2)
    acc += [test_model(model,feature2_sc,y2)]
print(acc)

In [ ]:
y_pred=model.predict(feature2_sc)
metrics.accuracy_score(y2,y_pred>0.5)

## RNN

In [92]:
files = np.array(df.columns)
X = []
Y = []
#sc = StandardScaler(with_mean=False)
i = 0
start = time.time()
for file in files:
    emg_data = pd.read_csv('./data/'+file)
    #emg_data = emg_data.fillna({'LEFT_TA':emg_data.LEFT_TA.mean(),
    #                            'LEFT_TS':emg_data.LEFT_TS.mean(),
    #                            'LEFT_BF':emg_data.LEFT_BF.mean(),
    #                            'LEFT_RF':emg_data.LEFT_RF.mean(),
    #                            'RIGHT_TA':emg_data.RIGHT_TA.mean(),
    #                            'RIGHT_TS':emg_data.RIGHT_TS.mean(),
    #                            'RIGHT_BF':emg_data.RIGHT_BF.mean(),
    #                            'RIGHT_RF':emg_data.RIGHT_RF.mean()})
    #emg_data = emg_data[emg_data.Label1 == emg_data.Label2].reset_index(drop=True)
    emg_data = emg_data.dropna().reset_index(drop=True)
    x = np.array(emg_data.iloc[:,3:])
    y = np.array(emg_data.Label2)
    #x,y = dp.generate_window_slide_data(emg_data,width=width,stride=stride,scaler=False,same_label=True)
    X += x.tolist()
    Y += y.tolist()
    i += 1
    print('i:%d'%i,x.shape)
end = time.time()
duration = end-start
print('Duration: %f'%(duration))

i:1 (16583, 8)
i:2 (4489, 8)
i:3 (35121, 8)
i:4 (31027, 8)
i:5 (30783, 8)
i:6 (159539, 8)
i:7 (225455, 8)
i:8 (362273, 8)
i:9 (28868, 8)
i:10 (20878, 8)
i:11 (25322, 8)
i:12 (24242, 8)
i:13 (21728, 8)
i:14 (27916, 8)
i:15 (27146, 8)
i:16 (25947, 8)
i:17 (26203, 8)
i:18 (89500, 8)
i:19 (70282, 8)
i:20 (79595, 8)
i:21 (73280, 8)
Duration: 4.539805


In [10]:
def train_model(model,feature,y,binary=True,file=None):
    if binary:
        ind = ((y==0)|(y==1)|(y==2)|(y==3)|(y==4)|(y==6))
        ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y.copy()
        #ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
        y_01[ind1] = 1
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:5}
    else:
        ind = ((y==1)|(y==2)|(y==6))
        #ind = ((y==1)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:5,1:1,2:1}#{0:2,1:1,2:10,3:2}
    x_full,x_test,y_full,y_test = train_test_split(np.array(feature)[ind,:],y_01,
                                                   test_size=0.2,
                                                   random_state=123,
                                                   shuffle=True)
    x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,
                                                       test_size=0.25,
                                                       random_state=555,
                                                       shuffle=True)
    
    #sm = BorderlineSMOTE(random_state=50,kind='borderline-1')
    #sm = SMOTE(random_state=50)
    #print(y_full.shape)
    #x_full,y_full = sm.fit_resample(x_full,y_full)
    #print(y_full_n.shape)
    x_full = np.reshape(x_full,(-1,128*8))
    sc = StandardScaler(with_mean=False)
    #sc = MinMaxScaler((0,1))
    x_train = sc.fit_transform(x_full)
    x_train = x_train.reshape((-1,128,8))
    #pca = PCA(n_components=150)
    #x_train = pca.fit_transform(x_train)
    #x_valid = sc.transform(x_valid)
    x_test = np.reshape(x_test,(-1,128*8))
    x_test = sc.transform(x_test)
    x_test = x_test.reshape((-1,128,8))
    #x_test = pca.transform(x_test)
    #x_train = x_full
    
    early_stopping = callbacks.EarlyStopping(patience = 10,
                                             monitor = 'val_accuracy', 
                                             restore_best_weights=True)
    history = model.fit(x_train,y_full,validation_data=[x_test,y_test],
                        epochs=300,batch_size=500,class_weight=cw,
                        callbacks=[early_stopping],
                        shuffle=True)
    
    if binary:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #test = metrics.accuracy_score(y_test,y_pred_t>0.5)
        
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(y_valid,np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))
        #train = metrics.accuracy_score(y_full,y_pred_ta>0.5)
        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
        
        #print('train: \n',metrics.confusion_matrix(y_full,y_pred_ta>0.5))
        #print('test: \n',metrics.confusion_matrix(y_test,y_pred_t>0.5))

    else:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
    print('test:%f'%test)
    #print('valid:%f'%valid)
    print('train:%f'%train)
    if file != None:
        model.save_model(file)
    return train,test,sc#,pca

In [ ]:
feature_re = feature_sc.reshape((-1,X.shape[1],1))
x_full,x_test,y_full,y_test = train_test_split(np.array(feature_re),np.array(y_01),test_size=0.2,random_state=123)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555)

In [11]:
input_ = layers.Input(shape=[128,8])
lstm1 = layers.LSTM(50,return_sequences=True)(input_)
drop1 = layers.Dropout(0.2)(lstm1)
lstm2 = layers.LSTM(50,return_sequences=True)(drop1)
drop2 = layers.Dropout(0.2)(lstm2)
#lstm3 = layers.LSTM(50,return_sequences=True)(drop2)
#drop3 = layers.Dropout(0.2)(lstm3)
lstm4 = layers.LSTM(50)(drop2)
drop4 = layers.Dropout(0.2)(lstm4)
dense1 = layers.Dense(64,activation='relu')(drop4)
drop5 = layers.Dropout(0.2)(dense1)
dense2 = layers.Dense(32,activation='relu')(drop5)
output = layers.Dense(3,activation='softmax')(dense2)
model = Model(inputs=[input_],outputs=[output])

In [12]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
early_stopping = callbacks.EarlyStopping(patience = 20,
                                         monitor = 'val_accuracy', 
                                         restore_best_weights=True)
history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                    epochs=100,batch_size=500,
                   callbacks=[early_stopping])

In [13]:
train,test,sc = train_model(model,np.array(x),np.array(y),False)

Train on 8872 samples, validate on 2219 samples
Epoch 1/300
8872/8872 [==============================] - 88s 10ms/sample - loss: 0.9841 - accuracy: 0.4876 - val_loss: 0.8857 - val_accuracy: 0.5002
Epoch 2/300
8872/8872 [==============================] - 82s 9ms/sample - loss: 0.8572 - accuracy: 0.5571 - val_loss: 0.8288 - val_accuracy: 0.6471
Epoch 3/300
8872/8872 [==============================] - 81s 9ms/sample - loss: 0.8051 - accuracy: 0.6443 - val_loss: 0.7573 - val_accuracy: 0.6670
Epoch 4/300
8872/8872 [==============================] - 81s 9ms/sample - loss: 0.7433 - accuracy: 0.6677 - val_loss: 0.7142 - val_accuracy: 0.6791
Epoch 5/300
8872/8872 [==============================] - 81s 9ms/sample - loss: 0.7212 - accuracy: 0.6787 - val_loss: 0.6996 - val_accuracy: 0.6886
Epoch 6/300
8872/8872 [==============================] - 87s 10ms/sample - loss: 0.7071 - accuracy: 0.6878 - val_loss: 0.6952 - val_accuracy: 0.7008
Epoch 7/300
8872/8872 [==============================] - 81s 9

KeyboardInterrupt: 

In [ ]:
feature2_re = feature2_sc.reshape((-1,feature.shape[1],1))

In [ ]:
test_model(model,x_test,y_test)

In [113]:
cost_matrix=np.array([[0,1,1,1],
                  [1,0,1,1],
                  [10,100,0,10],
                  [1.,1.,1,0]])
cost_matrix=np.array([[0,2.],
                  [5.,0]])

In [58]:
P0 = 55080/(55080+8181)
P1 = 8181/(55080+8181)
P = [P0,P1]
cost_vec = np.zeros(2)
for i in range(2):
    for j in range(2):
        if i == j:
            continue
        cost_vec[i]+=1/(1-P[i])*P[j]*cost_matrix[i,j]

In [59]:
cost_vec

array([2., 5.])

In [112]:
tf.transpose(t)

<tf.Tensor: id=1567100, shape=(4, 4), dtype=float64, numpy=
array([[  0.,   1.,  10.,   1.],
       [  1.,   0., 100.,   1.],
       [  1.,   1.,   0.,   1.],
       [  1.,   1.,  10.,   0.]])>

In [116]:
np.reshape(cost_matrix,(-1,1))

array([[0.],
       [2.],
       [5.],
       [0.]])